<a href="https://colab.research.google.com/github/Hanbin-git/kaggle/blob/main/notebookef924bbad8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

biniroun_car_image_path = kagglehub.dataset_download('biniroun/car-image')

print('Data source import complete.')


In [ ]:
# 필수 라이브러리 설치 (Kaggle Notebook 기본적으로 torch, timm 포함됨)
!pip install timm --quiet
!pip install albumentations --quiet


In [ ]:
import torch
import torch.nn as nn

# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# 모델 정의 (예: EfficientNet-B5)
import timm
model = timm.create_model('tf_efficientnet_b5', pretrained=True, num_classes=396)

# ✅ 멀티 GPU 적용 (DataParallel)
if torch.cuda.device_count() > 1:
    print(f"✅ Using {torch.cuda.device_count()} GPUs with DataParallel")
    model = nn.DataParallel(model)

# ✅ 모델을 device로 이동
model = model.to(device)


✅ Using device: cuda
✅ Using 2 GPUs with DataParallel


In [ ]:
import os

# /kaggle/input/car-image 내부 확인
print("✅ /kaggle/input/car-image 폴더 내용:", os.listdir('/kaggle/input/car-image'))


✅ /kaggle/input/car-image 폴더 내용: ['sample_submission.csv', 'test.csv', 'test', 'train']


In [ ]:
# Train 이미지 경로 확인
import glob

train_files = glob.glob('/kaggle/input/car-image/train/*/*.jpg')
print(f"✅ Train 이미지 수: {len(train_files)}")
print("샘플:", train_files[:3])

test_files = glob.glob('/kaggle/input/car-image/test/*.jpg')
print(f"✅ Test 이미지 수: {len(test_files)}")
print("샘플:", test_files[:3])

import pandas as pd

sample_submission = pd.read_csv('/kaggle/input/car-image/sample_submission.csv')
print(sample_submission.head())


✅ Train 이미지 수: 33137
샘플: ['/kaggle/input/car-image/train/3시리즈_G20_2019_2022/3시리즈_G20_2019_2022_0002.jpg', '/kaggle/input/car-image/train/3시리즈_G20_2019_2022/3시리즈_G20_2019_2022_0016.jpg', '/kaggle/input/car-image/train/3시리즈_G20_2019_2022/3시리즈_G20_2019_2022_0037.jpg']
✅ Test 이미지 수: 8258
샘플: ['/kaggle/input/car-image/test/TEST_04038.jpg', '/kaggle/input/car-image/test/TEST_07075.jpg', '/kaggle/input/car-image/test/TEST_04342.jpg']
           ID  1시리즈_F20_2013_2015  1시리즈_F20_2016_2019  1시리즈_F40_2020_2024  \
0  TEST_00000                   1                 0.0                 0.0   
1  TEST_00001                   1                 0.0                 0.0   
2  TEST_00002                   1                 0.0                 0.0   
3  TEST_00003                   1                 0.0                 0.0   
4  TEST_00004                   1                 0.0                 0.0   

   2008_2015_2017  2시리즈_그란쿠페_F44_2020_2024  2시리즈_액티브_투어러_F45_2019_2021  \
0             0.0               

In [ ]:
# label encoding 필요
import os

# 1️⃣ Train 디렉토리 경로
train_dir = '/kaggle/input/car-image/train'

# 2️⃣ 클래스명 리스트 (폴더명 기준 → 정렬)
class_names = sorted(os.listdir(train_dir))

# 3️⃣ 클래스 수 확인
num_classes = len(class_names)
print(f"✅ 클래스 수: {num_classes}")
print("샘플 클래스명:", class_names[:5])

# 4️⃣ 클래스명 → label (index) 매핑
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
idx_to_class = {idx: class_name for class_name, idx in class_to_idx.items()}

# 5️⃣ 샘플 출력 확인
print("\n✅ class_to_idx 샘플:")
for i, (k, v) in enumerate(class_to_idx.items()):
    print(f"{k} --> {v}")
    if i >= 4:
        break



✅ 클래스 수: 396
샘플 클래스명: ['1시리즈_F20_2013_2015', '1시리즈_F20_2016_2019', '1시리즈_F40_2020_2024', '2008_2015_2017', '2시리즈_그란쿠페_F44_2020_2024']

✅ class_to_idx 샘플:
1시리즈_F20_2013_2015 --> 0
1시리즈_F20_2016_2019 --> 1
1시리즈_F40_2020_2024 --> 2
2008_2015_2017 --> 3
2시리즈_그란쿠페_F44_2020_2024 --> 4


In [ ]:
import timm
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, use_aspect=False, use_color=False, num_classes=396):
        super(CustomModel, self).__init__()
        self.use_aspect = use_aspect
        self.use_color = use_color

        self.backbone = timm.create_model('efficientnet_b5', pretrained=True, num_classes=0)
        self.backbone_out_features = self.backbone.num_features

        aux_dim = 0
        if self.use_aspect:
            aux_dim += 1
        if self.use_color:
            aux_dim += 3

        self.fc = nn.Linear(self.backbone_out_features + aux_dim, num_classes)

    def forward(self, image, aspect_ratio=None, color_mean=None):
        x = self.backbone(image)

        aux_list = []
        if self.use_aspect:
            aux_list.append(aspect_ratio)
        if self.use_color:
            aux_list.append(color_mean)

        if aux_list:
            aux_features = torch.cat(aux_list, dim=1)
            x = torch.cat([x, aux_features], dim=1)

        out = self.fc(x)
        return out


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
import numpy as np
import os
import glob
from torchvision import transforms
from sklearn.model_selection import train_test_split

# 전체 JPG 파일 불러오기 (Train)
# ✅ Kaggle 경로
file_list = glob.glob('/kaggle/input/car-image/train/*/*.jpg')

# ✅ 클래스명 추출
def extract_class_name_jpg(path):
    return os.path.basename(os.path.dirname(path))

class_names = sorted(set(extract_class_name_jpg(f) for f in file_list))
class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

print(f"✅ 클래스 수: {len(class_to_idx)}")  # 396개 나와야 정상

# 라벨 생성
labels = [class_to_idx[extract_class_name_jpg(f)] for f in file_list]

train_files, val_files = train_test_split(
    file_list, test_size=0.1, stratify=labels, random_state=42
)

# Train / Val Split
print(f"✅ Train 파일 수: {len(train_files)}")
print(f"✅ Val 파일 수: {len(val_files)}")

# Transform 정의
# ✅ Train Transform
train_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ Val Transform
val_transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# 확장형 Dataset 버전
class CarImageDataset(Dataset):
    def __init__(self, file_list, class_to_idx, transform=None, use_aspect=False, use_color=False):
        self.file_list = file_list
        self.class_to_idx = class_to_idx
        self.transform = transform
        self.use_aspect = use_aspect
        self.use_color = use_color

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]

        # 🚗 Load Image (RGB 고정)
        image_pil = Image.open(path).convert("RGB")

        # 🚗 Feature: Aspect Ratio
        width, height = image_pil.size
        aspect_ratio = torch.tensor([width / height], dtype=torch.float32)

        # 🚗 Feature: Dominant Color (mean RGB)
        image_np = np.array(image_pil)
        color_mean = image_np.mean(axis=(0, 1)) / 255.0  # Normalize to 0~1
        color_mean = torch.tensor(color_mean, dtype=torch.float32)

        # 🚗 Transform
        if self.transform:
            image = self.transform(image_pil)
        else:
            image = transforms.ToTensor()(image_pil)

        # 🚗 Label
        class_name = extract_class_name_jpg(path)
        label = self.class_to_idx[class_name]

        # 🚗 Return mode
        if self.use_aspect and self.use_color:
            return image, aspect_ratio, color_mean, label
        elif self.use_aspect:
            return image, aspect_ratio, label
        elif self.use_color:
            return image, color_mean, label
        else:
            return image, label

# ✅ 전략 설정
USE_ASPECT = True    # 전략 B
USE_COLOR = False

# ✅ Dataset 정의 (전략 설정 값으로!)
train_dataset = CarImageDataset(train_files, class_to_idx, train_transform, use_aspect=USE_ASPECT, use_color=USE_COLOR)
val_dataset = CarImageDataset(val_files, class_to_idx, val_transform, use_aspect=USE_ASPECT, use_color=USE_COLOR)

# ✅ DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True, persistent_workers=True, prefetch_factor=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True, persistent_workers=True, prefetch_factor=2)

# ✅ Model 정의 (전략 설정 값으로!)
model = CustomModel(use_aspect=USE_ASPECT, use_color=USE_COLOR, num_classes=len(class_to_idx))
model = model.to(device)

# ✅ 테스트 출력
sample = next(iter(train_loader))
images, labels = sample[0], sample[-1]
print(f"✅ Batch 이미지 shape: {images.shape}")
print(f"✅ Batch label shape: {labels.shape}")




✅ 클래스 수: 396
✅ Train 파일 수: 29823
✅ Val 파일 수: 3314
✅ Batch 이미지 shape: torch.Size([16, 3, 384, 384])
✅ Batch label shape: torch.Size([16])


In [ ]:
# 학습용 optimizer / criterion 정의
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# 학습 epoch 설정
num_epochs = 10

# 학습 loop 실행 (내가 준 학습 loop 그대로 복붙 가능)


In [ ]:
import torch.optim as optim
import copy
from tqdm import tqdm

# ✅ device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ Model 생성 (전략 B → USE_ASPECT=True, USE_COLOR=False)
USE_ASPECT = True
USE_COLOR = False

model = CustomModel(use_aspect=USE_ASPECT, use_color=USE_COLOR, num_classes=len(class_to_idx))
model = model.to(device)

# ✅ Loss / Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# ✅ EarlyStopping 변수
best_val_loss = float('inf')
patience = 3
patience_counter = 0
best_model_wts = copy.deepcopy(model.state_dict())

# ✅ 학습 epoch 설정
num_epochs = 10

# ✅ 학습 loop 시작
for epoch in range(1, num_epochs + 1):
    print(f"\n🚀 Epoch {epoch}/{num_epochs}")

    # ✅ Clear GPU cache → 메모리 누수 방지
    torch.cuda.empty_cache()

    # === Train ===
    model.train()
    train_loss = 0.0
    train_correct = 0

    loop = tqdm(train_loader, desc=f"Train Epoch {epoch}", leave=False)
    for batch in loop:
        if USE_ASPECT and USE_COLOR:
            images, aspect_ratio, color_mean, labels = batch
            images, aspect_ratio, color_mean, labels = images.to(device), aspect_ratio.to(device), color_mean.to(device), labels.to(device)
            outputs = model(images, aspect_ratio, color_mean)
        elif USE_ASPECT:
            images, aspect_ratio, labels = batch
            images, aspect_ratio, labels = images.to(device), aspect_ratio.to(device), labels.to(device)
            outputs = model(images, aspect_ratio)
        elif USE_COLOR:
            images, color_mean, labels = batch
            images, color_mean, labels = images.to(device), color_mean.to(device), labels.to(device)
            outputs = model(images, color_mean=color_mean)
        else:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)
        train_correct += (outputs.argmax(1) == labels).sum().item()
        loop.set_postfix(loss=loss.item())

    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)

    # === Val ===
    model.eval()
    val_loss = 0.0
    val_correct = 0

    val_loop = tqdm(val_loader, desc=f"Valid Epoch {epoch}", leave=False)
    with torch.no_grad():
        for batch in val_loop:
            if USE_ASPECT and USE_COLOR:
                images, aspect_ratio, color_mean, labels = batch
                images, aspect_ratio, color_mean, labels = images.to(device), aspect_ratio.to(device), color_mean.to(device), labels.to(device)
                outputs = model(images, aspect_ratio, color_mean)
            elif USE_ASPECT:
                images, aspect_ratio, labels = batch
                images, aspect_ratio, labels = images.to(device), aspect_ratio.to(device), labels.to(device)
                outputs = model(images, aspect_ratio)
            elif USE_COLOR:
                images, color_mean, labels = batch
                images, color_mean, labels = images.to(device), color_mean.to(device), labels.to(device)
                outputs = model(images, color_mean=color_mean)
            else:
                images, labels = batch
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)

            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_loop.set_postfix(loss=loss.item())

    val_loss /= len(val_loader.dataset)
    val_acc = val_correct / len(val_loader.dataset)

    # === (선택) Validation 끝난 후에도 clear 가능
    torch.cuda.empty_cache()

    # === 로그 출력 ===
    print(f"✅ Epoch {epoch} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"✅ Epoch {epoch} | Val   Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}")

    # === EarlyStopping ===
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())
        save_path = f"/kaggle/working/EffNetB5_B_best_model.pth"
        torch.save(model.state_dict(), save_path)
        print(f"📦 Best model saved!")
        patience_counter = 0
    else:
        patience_counter += 1
        print(f"⚠️ EarlyStopping patience: {patience_counter}/{patience}")
        if patience_counter >= patience:
            print("⛔ Early stopping triggered.")
            break

# ✅ 학습 끝난 후 best model load
model.load_state_dict(best_model_wts)
print(f"\n✅ Best model loaded (Val Loss: {best_val_loss:.4f})")


✅ Using device: cuda

🚀 Epoch 1/10


Train Epoch 1:  39%|███▊      | 720/1864 [10:04<15:59,  1.19it/s, loss=1.11] 

In [ ]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import timm
from torchvision import transforms
from PIL import Image

# ✅ 고정 경로 (Kaggle)
TEST_DIR = "/kaggle/input/car-image/test"
SAMPLE_SUB_PATH = "/kaggle/input/car-image/sample_submission.csv"
NUM_CLASSES = 396

# ✅ 샘플 제출 파일에서 클래스명 추출
sample = pd.read_csv(SAMPLE_SUB_PATH)
column_names = sample.columns.tolist()[1:]  # 'ID' 제외

# ✅ Transform
transform = transforms.Compose([
    transforms.Resize((384, 384)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# ✅ 테스트용 Dataset
class TestJPGDataset(Dataset):
    def __init__(self, img_root, transform=None):
        self.file_list = []
        for file in os.listdir(img_root):
            if file.endswith('.jpg'):
                self.file_list.append(os.path.join(img_root, file))
        self.file_list.sort()  # 반드시 정렬

        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        path = self.file_list[idx]
        image = Image.open(path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        fname = os.path.basename(path).replace(".jpg", "")
        return image, fname

# ✅ DataLoader
test_dataset = TestJPGDataset(TEST_DIR, transform)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,   # Inference는 32~64 사용해도 괜찮음
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    prefetch_factor=2
)

# ✅ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# ✅ 전략 B Inference (fold 없이 단일 best model 사용 예시)
EXP_NAME = "B"

# ✅ 모델 경로 (Kaggle → working에서 불러오기 예시)
MODEL_PATH = f"/kaggle/working/EffNetB5_{EXP_NAME}_best_model.pth"

# ✅ CustomModel 불러와서 사용해야 함!
# (너가 학습 때 쓴 CustomModel 동일하게 넣어야 함)

class CustomModel(nn.Module):
    def __init__(self, use_aspect=False, use_color=False, num_classes=396):
        super(CustomModel, self).__init__()
        self.use_aspect = use_aspect
        self.use_color = use_color

        self.backbone = timm.create_model('efficientnet_b5', pretrained=False, num_classes=0)
        self.backbone_out_features = self.backbone.num_features

        aux_dim = 0
        if self.use_aspect:
            aux_dim += 1
        if self.use_color:
            aux_dim += 3

        self.fc = nn.Linear(self.backbone_out_features + aux_dim, num_classes)

    def forward(self, image, aspect_ratio=None, color_mean=None):
        x = self.backbone(image)

        aux_list = []
        if self.use_aspect:
            aux_list.append(aspect_ratio)
        if self.use_color:
            aux_list.append(color_mean)

        if aux_list:
            aux_features = torch.cat(aux_list, dim=1)
            x = torch.cat([x, aux_features], dim=1)

        out = self.fc(x)
        return out

# ✅ 모델 생성 & 로드
USE_ASPECT = True    # 전략 B
USE_COLOR = False

model = CustomModel(use_aspect=USE_ASPECT, use_color=USE_COLOR, num_classes=NUM_CLASSES)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# ✅ 추론 시작
all_probs = []

with torch.no_grad():
    for imgs, names in tqdm(test_loader, desc=f"🚀 Inference EXP {EXP_NAME}"):
        imgs = imgs.to(device)

        # 전략 B이므로 aspect_ratio 필요 없음 → 그냥 image만 사용
        outputs = model(imgs, aspect_ratio=None)
        probs = F.softmax(outputs, dim=1)
        all_probs.append(probs.cpu().numpy())

# ✅ 결과 정리
all_probs = np.concatenate(all_probs, axis=0)

results = []
for idx, path in enumerate(test_dataset.file_list):
    fname = os.path.basename(path).replace(".jpg", "")
    row = {"ID": fname}
    row.update({class_name: all_probs[idx, i] for i, class_name in enumerate(column_names)})
    results.append(row)

submission_df = pd.DataFrame(results)
submission_df = submission_df[["ID"] + column_names]

# ✅ 파일 저장 (Kaggle working에 저장!)
SAVE_SUBMISSION_PATH = f"/kaggle/working/submission_B.csv"
submission_df.to_csv(SAVE_SUBMISSION_PATH, index=False)

print(f"\n✅ Submission 저장 완료: {SAVE_SUBMISSION_PATH}")
